In [1]:
%load_ext autoreload
%autoreload 2

from nn import EncoderRNN, DecoderRNN, Seq2Seq
import utils
import cPickle as pickle
import torch
from tqdm import tqdm
import random
# import tensorflow as tf

In [2]:
lang, pairs = utils.prepare_data('../data/cmd/dialogs.txt')
# lang, pairs = utils.prepare_data('../res/friends.txt', persona=True)
# lang, pairs = pickle.load(open('data.pkl'))

Reading lines...
Done


In [3]:
pickle.dump((lang, pairs), open('data.pkl', 'wb'))

# Testing if everything works

In [7]:
hidden_size = 10
enc_size = hidden_size
dec_size = hidden_size
learning_rate = 0.01
emb_dims = 5
max_length = 11
persona_size = 4
lang.max_length = max_length
seq2seq = Seq2Seq(lang, enc_size, dec_size, emb_dims, max_length, learning_rate, 
                  persona_size=persona_size, attention=False, persona=False)

In [15]:
from tqdm import tqdm
test_pair = [('Joey:how are you', 'Chandler:pretty good'), ('Chandler:what are you doing', 'Joey:nothing')]
# test_pair = [('how are you', 'pretty good'),
#              ('write some shit with shit seven eight nine ten as a dog', 'you talked a lot')]
for i in tqdm(xrange(500)):
    _, loss = seq2seq.forward(test_pair)

100%|██████████| 500/500 [00:03<00:00, 150.17it/s]


In [16]:
# test_pair = [('how are you', ''), ('what are you', '')]
test_pair = [('Joey:how are you', 'Chandler:'), ('Chandler:what are you doing', 'Joey:')]
output, _ = seq2seq.forward(test_pair, False)
print output

['nothing', 'nothing']


In [16]:
seq2seq.encoder.rnn.set_weights({'all_weights':d_weights})

NameError: name 'd_weights' is not defined

# Training the model

In [23]:
hidden_size = 512
enc_size = hidden_size
dec_size = hidden_size
learning_rate = 0.01
emb_dims = 200
max_length = 30
lang.max_length = max_length
seq2seq = Seq2Seq(lang, enc_size, dec_size, emb_dims, max_length, learning_rate, 
                  persona_size=20, attention=True, persona=False)

In [24]:
seq2seq.load_model()

In [4]:
N = 32 # Max limit for attention model
buckets = []
for i in xrange(0, len(pairs) / N, N):
    buckets.append(pairs[i:min(i+N, len(pairs) - 1)])

In [5]:
import random

for i in tqdm(xrange(20000)):
    r_pair = random.choice(buckets)
    _, loss = seq2seq.forward(r_pair)

 91%|█████████▏| 18284/20000 [6:58:09<39:50,  1.39s/it]

KeyboardInterrupt: 

In [23]:
seq2seq.save_model()

In [129]:
output, _ = seq2seq.forward([('hi', ''), ('what are you doing', '')], False)

In [130]:
output

['why im really UNK you', 'UNK you you']

# Pretrained weights

## Dump weights

In [7]:
import cPickle as pickle
# set the same weights for encoder, only change is in the decoder
# dump the weights from seq2seq model
pretrained_encoder_weights = seq2seq.encoder.rnn.all_weights
pretrained_decoder_weights = seq2seq.decoder.rnn.all_weights

pickle.dump(pretrained_encoder_weights, open('../models/pretrained_encoder_weights', 'wb'))
pickle.dump(pretrained_decoder_weights, open('../models/pretrained_decoder_weights', 'wb'))

## Load weights

In [14]:
from torch.autograd import Variable
import torch.cuda as cuda
import cPickle as pickle

load_to_persona = False

pretrained_encoder_weights = pickle.load(open('../models/pretrained_encoder_weights'))
pretrained_decoder_weights = pickle.load(open('../models/pretrained_decoder_weights'))

if load_to_persona is True:
    temp_var = Variable(cuda.FloatTensor(4*hidden_size, emb_dims + hidden_size + persona_size))
    temp_var[:, :(emb_dims + hidden_size)] = pretrained_decoder_weights[0][0]
decoder_weights = seq2seq.decoder.rnn.all_weights
encoder_weights = seq2seq.encoder.rnn.all_weights
decoder_weights = pretrained_decoder_weights
if load_to_persona is True:
    decoder_weights[0][0] = temp_var
encoder_weights = pretrained_encoder_weights

seq2seq.decoder.rnn.set_weights(pretrained_decoder_weights)
seq2seq.encoder.rnn.set_weights(pretrained_encoder_weights)